In [ ]:
import xml.etree.ElementTree as ET

def create_dataset(origin):
    with open(origin) as input:
        n_line=0
        temp_file = open('xmls/temp/temp.xml','w')
        while True:
            line = input.readline()
            if not line: 
                break
            if line.startswith('<?xml') and n_line!=0:
                temp_file.close()
                tree = ET.parse('xmls/temp/temp.xml')
                us_patent_grant = tree.getroot()
                for publication_reference in us_patent_grant.iter("publication-reference"):
                    print("patent number: "+publication_reference.find("document-id").find("doc-number").text)
                for us_bibliographic_data_grant in us_patent_grant.iter("us-bibliographic-data-grant"):
                    title=us_bibliographic_data_grant.find("invention-title").text
                    if title: print("title: "+title)
                for abstract in us_patent_grant.iter("abstract"):
                    print("abstract: "+abstract.find("p").text)
                temp_file = open('xmls/temp/temp.xml','w')
            temp_file.write(line)   
            n_line+=1

In [47]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import zipfile
import os
 
BulkDataStorageSystem_url = 'https://bulkdata.uspto.gov/'
BulkDataStorageSystem = requests.get(BulkDataStorageSystem_url)
BulkDataStorageSystem_soup = BeautifulSoup(BulkDataStorageSystem.text, 'html.parser')
for link in BulkDataStorageSystem_soup.find_all('a'):
    if str(link.get_text()).startswith("Patent Grant Full Text Data"):
        print("downloading patents from "+str(link.get('href'))[-4:])
        PatentGrantFullTextData=requests.get(link.get('href'))
        PatentGrantFullTextData_soup = BeautifulSoup(PatentGrantFullTextData.text, 'html.parser')
        for table in PatentGrantFullTextData_soup.find_all('table'):
            for zip_downloader in tqdm(table.find_all('a'), total= len(table.find_all('a')), ):
                if str(zip_downloader.get_text()).startswith("ipg"):
                    print(str(link.get('href'))+"/"+str(zip_downloader.get('href')))
                    download=requests.get(str(link.get('href'))+"/"+str(zip_downloader.get('href')))
                    with open("./downloaded_file.zip","wb") as downloaded_file:
                        downloaded_file.write(download.content)
                        with zipfile.ZipFile("./downloaded_file.zip", 'r') as zip:
                            zip.extract(zip.namelist()[0])
                    create_dataset(zip.namelist()[0])
                    os.remove(zip.namelist()[0])

downloading patents from 2023


0it [00:00, ?it/s]
  0%|          | 0/21 [00:00<?, ?it/s]

https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/2023/ipg230103.zip


  0%|          | 0/21 [03:30<?, ?it/s]


NameError: name 'create_dataset' is not defined